<a href="https://colab.research.google.com/github/Glasiermedic/Natural-Language-process/blob/master/nlp_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report

from IPython.display import display

import spacy
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk
nltk.download('gutenberg')

from spacy.lang.en import English

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import LabelEncoder, Imputer
from sklearn.model_selection import train_test_split

from timeit import default_timer as timer

import os

import pydotplus
from sklearn import tree
from sklearn import preprocessing
%matplotlib inline
sns.set_style('white')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
#from nltk.corpus import gutenberg, stopwords


In [0]:
#print (gutenberg.fileids())

Time to bag some words! Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur. We will exclude stopwords and punctuation. In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [0]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [6]:
%%time
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

CPU times: user 2.12 s, sys: 216 ms, total: 2.34 s
Wall time: 2.34 s


In [7]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head(20)

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll
5,"(I, shall, be, late, !, ')",Carroll
6,"((, when, she, thought, it, over, afterwards, ...",Carroll
7,"(A, WATCH, OUT, OF, ITS, WAISTCOAT, -, POCKET,...",Carroll
8,"(,, Alice, started, to, her, feet, ,, for, it,...",Carroll
9,"(In, another, moment, down, went, Alice, after...",Carroll


Time to bag some words! Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur. We will exclude stopwords and punctuation. In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [0]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [9]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,market,indifference,little,unite,public,line,listen,accidentally,St,ball,decide,milk,furniture,seen,go,acquaint,confirm,charity,faculty,volume,comprehensive,night,child,history,agreeable,...,Trafalgar,matter,secret,acceptable,ah,hair,world,aggravation,house,sorry,offensive,application,picture,exceed,near,grandson,acre,attraction,power,true,liberality,charm,lavish,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


###Trying out BoW
####Now let's give the bag of words features a whirl by trying a random forest.

In [10]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=1)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9736842105263158

Test set score: 0.8314606741573034


In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', solver = 'lbfgs', max_iter = 1000, C = 100) # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(266, 1612) (266,)
Training set score: 0.9849624060150376

Test set score: 0.8707865168539326


In [12]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9774436090225563

Test set score: 0.8089887640449438


In [13]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [0]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [0]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [16]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
Processing row 150
done


In [17]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6341463414634146


col_0,Austen,Carroll
row_0,,
Austen,136,34
Carroll,56,20


### Challenge 0:
#### Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
kfolds = StratifiedShuffleSplit(n_splits =10, random_state=1)
from sklearn.model_selection import GridSearchCV

In [19]:
word_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Columns: 1614 entries, market to text_source
dtypes: object(1614)
memory usage: 5.5+ MB


In [0]:
word_counts2 = word_counts.copy()

In [21]:
word_counts2.head()

,market,indifference,little,unite,public,line,listen,accidentally,St,ball,decide,milk,furniture,seen,go,acquaint,confirm,charity,faculty,volume,comprehensive,night,child,history,agreeable,...,Trafalgar,matter,secret,acceptable,ah,hair,world,aggravation,house,sorry,offensive,application,picture,exceed,near,grandson,acre,attraction,power,true,liberality,charm,lavish,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [22]:
word_counts2['sents_len'] = 0
word_counts2['punct_ratio'] = 0
word_counts2['stop_ratio'] = 0
word_counts2['word_len'] = 0

for i, sentence in enumerate(word_counts2['text_sentence']):
    punct_count, stop_count = 0, 0
    for word in word_counts2.loc[i, 'text_sentence']:
        #If a column doesn't exist for the pos_, create it.
        if word_counts2.columns.isin([word.pos_]).sum()==0:
            word_counts2[word.pos_] = 0
        word_counts2.loc[i, word.pos_] += 1
        punct_count += word.is_punct
        stop_count += word.is_stop
    word_counts2.loc[i, 'sents_len'] = len(sentence)
    word_counts2.loc[i, 'punct_ratio'] = punct_count/len(sentence)
    word_counts2.loc[i, 'stop_ratio'] = stop_count/len(sentence)
    word_counts2.loc[i, 'word_len'] = len(str(sentence))/len(sentence)
    word_counts2.iloc[i, 3068:] /= len(sentence)
    
    if i % 50 == 0:
        print(i)

0
50
100
150
200
250
300
350
400


In [0]:
Y = word_counts2['text_source']
X = np.array(word_counts2.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

In [24]:
lr_params = {'C': [0.0001, 0.01, 1, 100, 10000]}
lr_grid = GridSearchCV(lr, lr_params, cv=kfolds)
lr_grid.fit(X_train, y_train)
print(lr_grid.best_params_)
print(lr_grid.best_score_)

{'C': 100}
0.8592592592592593


In [25]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', solver = 'lbfgs', max_iter = 1000, C = 100) # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print(cross_val_score(lr, X_train, y_train, cv=kfolds).mean())  




(266, 1630) (266,)
Training set score: 0.9924812030075187

Test set score: 0.898876404494382
0.8592592592592592


In [26]:
(110+31)/(110+22+15+31)


0.7921348314606742

### Challenge 1:
#### Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [27]:
# Clean the moby_dick data.
moby = gutenberg.raw('melville-moby_dick.txt')
moby = re.sub(r'VOLUME \w+', '', moby)
moby = re.sub(r'CHAPTER \w+', '', moby)
moby = text_cleaner(moby[:int(len(moby)/60)])
print(moby[:100])

ETYMOLOGY. (Supplied by a Late Consumptive Usher to a Grammar School) The pale Usher threadbare in c


In [0]:
# Parse our cleaned data.
moby_doc = nlp(moby)

In [0]:
# Group into sentences.
moby_sents = [[sent, 'melville'] for sent in moby_doc.sents]
moby_sents = moby_sents[:len(alice_sents)]

In [30]:
moby_ali_sentences = pd.DataFrame(alice_sents + moby_sents)
moby_per_sentences = pd.DataFrame(moby_sents + persuasion_sents)

#We'll keep the same common word features so our model will run.
moby_ali_bow = bow_features(moby_ali_sentences, common_words)
moby_per_bow = bow_features(moby_per_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
done


In [0]:
def add_features(df):

    df['sents_len'] = 0
    df['punct_ratio'] = 0
    df['stop_ratio'] = 0
    df['word_len'] = 0

    for i, sentence in enumerate(df['text_sentence']):
        punct_count, stop_count = 0, 0
        for word in df.loc[i, 'text_sentence']:
            #If a column doesn't exist for the pos_, create it.
            if df.columns.isin([word.pos_]).sum()==0:
                df[word.pos_] = 0
            df.loc[i, word.pos_] += 1
            punct_count += word.is_punct
            stop_count += word.is_stop
        df.loc[i, 'sents_len'] = len(sentence)
        df.loc[i, 'punct_ratio'] = punct_count/len(sentence)
        df.loc[i, 'stop_ratio'] = stop_count/len(sentence)
        df.loc[i, 'word_len'] = len(str(sentence))/len(sentence)
        df.iloc[i, 3068:] /= len(sentence)
    
        if i % 500 == 0:
            print(i)
        
    return df

In [32]:
df_moby_ali = add_features(moby_ali_bow)

0


In [33]:
df_moby_per = add_features(moby_per_bow)

0


In [34]:
# Now we can model it!
# Let's use logistic regression again.
df = df_moby_ali.copy()

Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', solver = 'lbfgs', max_iter = 1000, C = 100) # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print(cross_val_score(lr, X_train, y_train, cv=kfolds).mean())  


(154, 1631) (154,)
Training set score: 0.9935064935064936

Test set score: 0.8076923076923077
0.775


In [40]:
from sklearn.model_selection import cross_val_predict
# Now we can model it!
# Let's use logistic regression again.
from sklearn.metrics import roc_curve, auc, f1_score, recall_score, precision_score
#recall_average = recall_score(y_test, y_predict, average="binary", pos_label="neg")

df1 =df_moby_ali.copy()
df = df_moby_per.copy()

Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

Y1 = df1['text_source']
X1 = np.array(df1.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.4, random_state=0)

print(cross_val_score(lr,X_train, y_train, cv=kfolds, verbose = 2))
pred_y_sklearn =cross_val_predict(lr, X_test, y_test, cv=10)
y_true = y_test
y_pred = cross_val_predict(lr, X_test, y_test, cv=10)
print('\n Austin Confusion Matrix')
#print(\n confusion_matrix(y_pred, y_true))
print(pd.crosstab(pred_y_sklearn, y_test))
print("")
print(classification_report(y_pred, y_true))

print(cross_val_score(lr,X_train1, y_train1, cv=kfolds, verbose = 2))
pred_y_sklearn1 =cross_val_predict(lr, X_test1, y_test1, cv=10)
y_true1 = y_test1
y_pred1 = cross_val_predict(lr, X_test1, y_test1, cv=10)
print('\n Carroll Confusion Matrix')
#print(\n confusion_matrix(y_pred, y_true))
print(pd.crosstab(pred_y_sklearn1, y_test1))
print("")
print(classification_report(y_pred1, y_true1))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=   0.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ................................................. , total=   0.4s
[CV]  ................................................................
[CV] ................................................. , total=   0.4s
[CV]  ................................................................
[CV] ................................................. , total=   0.4s
[CV]  ................................................................
[CV] ................................................. , total=   0.4s
[CV]  ................................................................
[CV] ................................................. , total=   0.4s
[CV]  ................................................................
[CV] ................................................. , total=   0.4s
[CV]  ................................................................
[CV] ................................................. , total=   0.4s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    4.0s finished


...

[CV] ................................................. , total=   0.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ................................................. , total=   0.3s
[CV]  ................................................................
[CV] ................................................. , total=   0.3s
[CV]  ................................................................
[CV] ................................................. , total=   0.3s
[CV]  ................................................................
[CV] ................................................. , total=   0.3s
[CV]  ................................................................
[CV] ................................................. , total=   0.2s
[CV]  ................................................................
[CV] ................................................. , total=   0.2s
[CV]  ................................................................
[CV] ................................................. , total=   0.3s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.6s finished



 Carroll Confusion Matrix
text_source  Carroll  melville
row_0                         
Carroll           42         8
melville          11        43

              precision    recall  f1-score   support

     Carroll       0.79      0.84      0.82        50
    melville       0.84      0.80      0.82        54

    accuracy                           0.82       104
   macro avg       0.82      0.82      0.82       104
weighted avg       0.82      0.82      0.82       104



#### When it comes to comparing Austin and Melville we get an F1 score for Austin of 84% and for Melville of only 66% I would trust our model less based on this result when it comes to other authors not used in the training.  I would probably suggest diversitfying the training sample to include more varieties of authors so we can create a more robust unique token list or use more advanced models or incorporate prefix, suffix, sentiment and other features to determine if they would add to the accuracy.  When we look at  Carroll compared to Melville the F1 score for both authors is 82%.  Based on these results I would say our model is effective at making the prediction a majority of the time but again I would want to improve the accuracy to over 95% using some more advanced optimization techiniques. 